In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pydicom
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.measure import marching_cubes

In [2]:

def segment_knee_image(knee_img):
    # Apply Otsu's thresholding to segment knee
    threshold_value = threshold_otsu(knee_img)
    knee_mask = knee_img > threshold_value

    # Label connected components
    labeled_mask = label(knee_mask)

    # Get properties of connected components
    regions = regionprops(labeled_mask)

    # Find the largest connected component (knee area)
    knee_area = None
    max_area = 0
    for region in regions:
        if region.area > max_area:
            max_area = region.area
            knee_area = region

    # Create a binary mask for the largest connected component (knee)
    knee_mask = np.zeros_like(labeled_mask)
    knee_mask[labeled_mask == knee_area.label] = 1

    # Remove everything else in the image outside the knee mask
    segmented_knee_img = knee_img * knee_mask

    return segmented_knee_img

In [3]:
input_dir = './Knee_dataset/series-00002/'
output_dir = './KneeOutput/'

# Create the output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Get a list of all files in the input directory
dicom_files = [file for file in os.listdir(input_dir) if file.endswith('.dcm')]

# Process each knee image
for file_name in dicom_files:
    # Load the DICOM image
    dicom_data = pydicom.dcmread(os.path.join(input_dir, file_name))
    knee_img = dicom_data.pixel_array

    # Segment the knee image
    segmented_knee_img = segment_knee_image(knee_img)

    # Save the segmented knee image
    output_file_path = os.path.join(output_dir, f'segmented_knee_{file_name.replace(".dcm", ".png")}')
    plt.imsave(output_file_path, segmented_knee_img, cmap='gray')

In [4]:
# Define the spacing between slices
slice_spacing = 5  # You can adjust this value based on your preference and dataset characteristics

segmented_knee_images = []

# Process each knee image and store the segmented images with spacing in the list
for file_name in dicom_files:
    # Load the DICOM image
    dicom_data = pydicom.dcmread(os.path.join(input_dir, file_name))
    knee_img = dicom_data.pixel_array

    # Segment the knee image
    segmented_knee_img = segment_knee_image(knee_img)
    segmented_knee_images.append(segmented_knee_img)

    # Add empty slices as spacing
    for _ in range(slice_spacing):
        segmented_knee_images.append(np.zeros_like(segmented_knee_img))

# Stack segmented knee images into a 3D volume
stacked_volume = np.stack(segmented_knee_images, axis=-1)

# Apply marching cubes algorithm to extract mesh vertices and faces
vertices, faces, _, _ = marching_cubes(stacked_volume,  level=0.3,spacing=(0.5, 0.5, 0.5), step_size=2)

# Define a function to save the mesh as an OBJ file
def save_mesh(vertices, faces, filename):
    with open(filename, 'w') as f:
        for v in vertices:
            f.write(f"v {v[0]} {v[1]} {v[2]}\n")
        for face in faces:
            f.write(f"f {face[0]+1} {face[1]+1} {face[2]+1}\n")  # Add 1 to each index to match OBJ format

# Specify the output directory and filename for the OBJ mesh
obj_output_dir = './MeshOutput/'
os.makedirs(obj_output_dir, exist_ok=True)
obj_filename = os.path.join(obj_output_dir, 'knee_mesh.obj')

# Save the mesh as an OBJ file
save_mesh(vertices, faces, obj_filename)
